# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import re
import pickle
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import warnings
!pip install numpy --upgrade
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Requirement already up-to-date: numpy in /opt/conda/lib/python3.6/site-packages (1.19.5)


In [22]:
import sqlite3 
con = sqlite3.connect("DisasterResponse10.db")
cursor = con.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('message_and_categories_ds10',)]


In [23]:
# load data from database
#df = cursor.execute("SELECT * FROM message_and_categories_ds4 ")
#engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('message_and_categories_ds10', con = 'sqlite:///DisasterResponse10.db')
df.to_csv('ML_Data.csv')
df.head()
X = df.loc[:,["message"]]
X = X.squeeze()
Y = df.iloc[:,list(range(36))]
for column in Y.columns:
    print(column, ': ',  Y[column].unique())
print(X)
print(Y)

related :  ['1' '0']
request :  ['0' '1']
offer :  ['0' '1']
aid_related :  ['0' '1']
medical_help :  ['0' '1']
medical_products :  ['0' '1']
search_and_rescue :  ['0' '1']
security :  ['0' '1']
military :  ['0' '1']
child_alone :  ['0']
water :  ['0' '1']
food :  ['0' '1']
shelter :  ['0' '1']
clothing :  ['0' '1']
money :  ['0' '1']
missing_people :  ['0' '1']
refugees :  ['0' '1']
death :  ['0' '1']
other_aid :  ['0' '1']
infrastructure_related :  ['0' '1']
transport :  ['0' '1']
buildings :  ['0' '1']
electricity :  ['0' '1']
tools :  ['0' '1']
hospitals :  ['0' '1']
shops :  ['0' '1']
aid_centers :  ['0' '1']
other_infrastructure :  ['0' '1']
weather_related :  ['0' '1']
floods :  ['0' '1']
storm :  ['0' '1']
fire :  ['0' '1']
earthquake :  ['0' '1']
cold :  ['0' '1']
other_weather :  ['0' '1']
direct_report :  ['0' '1']
0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someo

### 2. Write a tokenization function to process your text data

def tokenize(text):
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
 # Stem word tokens and remove stop words
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return stemmed




In [24]:
def tokenize(text):
    """ Normalize text string, tokenize text string and remove stop words from text string
    Args: 
        Text string with message
    Returns 
        Normalized text string with word tokens 

"""
    
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    clean_tokens = []
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return stemmed

def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [25]:
pipeline = Pipeline([
    
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('Tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))

])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1 )
print(x_train)

23330    This was not only in preparation for expected ...
13120    Saudi Public Assistance for Pakisatan Earthqua...
11890    blackout in chile. nearly the whole country. s...
12713    Baldwin Center in Stratford. Is without power ...
11660    The ISS will pass over Santiago at mag 0.9 at ...
8729     Each one hour ask favour, pardon, mercy,for ha...
6587     I don't find nothing: covers and so one.Can yo...
13973    The immigrant populations are likely to be hig...
5647                   I want to know the info about 4636 
20940    Parents hesitated to send children to school o...
25720    As of this year, radical groups now controlled...
10731    Wand to donate jackets , hot meals , non-peris...
13689    The floods, which began in late July after hea...
16422    With flood-hit families whose houses were tota...
3406     Hospital Medecin Sans Frontiers (Doctors witho...
17348    His club and others in the Eastern High Sierra...
7930        what problem who contribute at a earth quake

In [ ]:
pipeline.fit(x_train, y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average = 'weighted')
        recall = recall_score(actual[:, i], predicted[:, i], average ='weighted')
        f1 = f1_score(actual[:, i], predicted[:, i], average = 'weighted')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df 

In [ ]:
# Calculate evaluation metrics for training set
y_train_pred = pipeline.predict(x_train)
col_names = list(Y.columns.values)



In [ ]:
print(get_eval_metrics(np.array(y_train), y_train_pred, col_names))

In [ ]:
# Calculate evaluation metrics for test set
y_test_pred = pipeline.predict(x_test)

eval_metrics0 = get_eval_metrics(np.array(y_test), y_test_pred, col_names)
print(eval_metrics0)

In [ ]:
# Calculation the proportion of each column that have label == 1
#Y.sum()/len(Y)
#print(Y.sum())
#print(len(Y))
#type(Y)
#(Y>2).count()
#Y[Y >2].count()

Y

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers
    
    Args:
    y_true: array. Array containing actual labels.
    y_pred: array. Array containing predicted labels.
        
    Returns:
    score: float. Median F1 score for all of the output classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [ ]:
# Create grid search object
## Parameters for random forest
'''

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(performance_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10)
'''

In [ ]:
pipeline.get_params().keys()

In [ ]:
# Create grid search object
# Parameters for Decision Tree
parameters = {'vect__min_df': [1, 5],
              'Tfidf__use_idf':[True, False],
              'clf__estimator__criterion':[2, 5]}

scorer = make_scorer(performance_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10, n_jobs = -1)

In [ ]:
# Find best parameters
np.random.seed(81)
tuned_model = cv.fit(x_train, y_train)

In [ ]:
# Get results of grid search
tuned_model.cv_results_

In [ ]:
# Best mean test score
np.max(tuned_model.cv_results_['mean_test_score'])

In [ ]:
# Parameters for best mean test score
tuned_model.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Calculate evaluation metrics for test set
tuned_pred_test = tuned_model.predict(X_test)

eval_metrics1 = get_eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics1)

In [ ]:
# Get summary stats for first model
eval_metrics0.describe()

In [ ]:
# Get summary stats for tuned model
eval_metrics1.describe()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Try using SVM instead of Random Forest Classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [1, 2, 3],
              'clf__estimator__C':[1, 10, 100]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(81)
tuned_model2 = cv2.fit(X_train, Y_train)

In [ ]:
# Get results of grid search
tuned_model2.cv_results_

In [ ]:
# Calculate evaluation metrics for test set
tuned_pred_test2 = tuned_model2.predict(X_test)

eval_metrics2 = get_eval_metrics(np.array(Y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

### 9. Export your model as a pickle file

In [ ]:
# Pickle best model
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.